In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, load_prompt
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain_core.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
import sqlite3
import time

In [6]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.05,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [7]:
example_answer = [
    {
        "question": """
아래는 대학영어 강의에 대한 강의평이야. 강의평을 요약해.
1. 정말 좋은 교수님과 너무 고등학생 같은 시험.. 꼼꼼히 공부해야 합니다!
2. 설명 잘 해주시고 시험은 내신처럼 나옵니다. 꼼꼼히 지문 보세요.
3. 설명 잘해주시고 고등학교 영어 내신처럼 공부하면 됩니다
4. 교수님께서 설명을 친절하게 해주십니다. 2인1조로 구성되어 매수업 해야할 과제가 있지만 강의시간 안에 끝나서 부담이 없습니다. 정말 좋은 교수님이라 추천합니다
5. 기출문제 주십니다. 매 수업 팀 과제가 귀찮아요.  (2인 1조) 쁠은 다 채워주셔요.
""",
        "answer": """
대체로 교수님의 강의력에 대한 긍정적인 평가가 많아요. 친절하고 상세한 설명으로 학생들의 이해를 돕고, 기출문제 제공 등 학습에 도움이 되는 부분도 있어요. 시험은 고등학교 내신처럼 꼼꼼한 암기와 지문 분석을 요구하며, 수업 시간 내에 완료 가능한 2인 1조 팀 과제가 매 수업 있어요. 전반적으로는 좋은 강의이지만, 시험 준비를 철저히 해야 해요.
""",
    },
    {
        "question": """
아래는 광운인되기 강의에 대한 강의평이야. 강의평을 요약해.
1. 강의만 온라인으로 다 들으면 pass주는 과목입니다. 1학점 챙겨가세요
2. 온라인강의 꿀! 학기 말에 몰아서 하면 됩니다!!
3. 강의만 수강하시면 패스 충분히 받습니다. 온라인 강의라 부담이 없습니다
4. 강의 제때제때 틀어놓는 거랑 공지사항 내용 꼼꼼히 확인하기만 하면 쉽게 패스 가능합니다
""",
        "answer": """
광운인되기 강의는 온라인 강의로, 강의만 이수하면 학점 취득이 용이하다는 내용의 긍정적인 평가가 대부분이에요. 학기말에 몰아서 수강해도 무방하며, 강의 시청과 공지사항 확인만으로도 쉽게 패스할 수 있다는 의견이 많아요.
""",
    },
    {
        "question": """
아래는 유기화학1 강의에 대한 강의평이야. 강의평을 요약해.
1. 교수님 강의력 좋은 편은 아님.. 족보 타는 강의 기말은 많이는 안탄듯 가지고 있으면 좋음
2. 족보 너무 많이 탑니다 ㅠ 족보 무조건 구해서 시험 보시는 거 추천드립니다
3. 과제, 족보 이 두개만 풀면 시험 무난하게 볼 수 있어요. 근데 성적을 잘 주시는 편은 아닌 것 같습니다
""",
        "answer": """
유기화학1 강의는 교수님의 강의력이 부족하다는 평가와 함께, 시험에서 족보의 의존도가 매우 높다는 점이 강조되고 있어요. 과제와 족보만 충분히 준비하면 시험을 무난히 볼 수 있지만, 성적 분포는 엄격한 편이에요. 족보 획득이 고득점의 관건이며, 강의 내용보다는 족보 활용에 집중해야 해요.
""",
    },
]

In [ ]:
conn = sqlite3.connect('./kwu-lecture-recommendation-v8.db')
cursor = conn.cursor()


cursor.execute("SELECT DISTINCT lectureID FROM LectureReview")
lecture_ids = cursor.fetchall()


for (lecture_id,) in lecture_ids:
    start_time = time.time()

    cursor.execute(
        """
        SELECT lr.review, ll.lecName 
        FROM LectureReview lr
        JOIN LectureList ll ON lr.lectureID = ll.lectureID
        WHERE lr.lectureID = ? 
        ORDER BY lr.reviewer DESC 
        LIMIT 50;
        """,
        (lecture_id,)
    )

    reviews = cursor.fetchall()

    review_texts = [review[0] for review in reviews]
    lecName = reviews[0][1] if reviews else None

    if len(review_texts) < 3:
        print("| pass | len(reviews) < 3")
        continue

    output = "\n".join(review_texts)

    print(f"| info | in {lecName} - {lecture_id}")

    question = f"""
    아래는 {lecName} 강의에 대한 강의평이야. 강의평을 요약해.
    {output}
    """

    example_prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "{question}"),
            ("ai", "{answer}"),
        ]
    )

    example_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=example_answer,
    )

    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "너는 강의를 선택하는 학생들을 위해 강의 리뷰를 요약하는 봇이야."),
            example_prompt,
            ("human", "강의 리뷰 리스트들을 함축해서 최종적으로 최소 한 줄, 최대 세 줄의 답변만을 내놓아야 해. 여러 리뷰들에서 중복되는 강의의 특징이 나타나거나, 필수적이라고 판단되는 부분만을 한국어로 요약해. 요약 외에는 어떠한 답변도 더하지 마.\n" + question),
        ]
    )

    chain = final_prompt | llm

    answer = chain.invoke({"question": question})

    final_answer = answer.content

    cursor.execute('''
        UPDATE LectureEverytimeData
        SET reviewSummary = ?
        WHERE lectureID = ?
    ''', (final_answer, lecture_id))
    conn.commit()

    if len(final_answer) > 130:
        final_answer = final_answer[:130] + '...'

    print(f"| good | {lecture_id}\t {lecName} \t | {final_answer}")
    print("|      |")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(
        f"| time | lectureID: {lecture_id}, elapsed time: {elapsed_time:.2f} seconds |")

conn.close()